<a id="coastal_change_classifier_top"></a>

# Coastal Change

*****
### This Notebook:  

- Highlights a simple method for detecting coastal change between two timeframes
- Visualizes water classification for two timeframes
- Visualizes water classification changes between timeframes
------  
### Motivation:  

International agencies like World Bank, UNEP, and USAID are currently reporting and addressing the problem of coastal erosion near Lomé, Togo.  The links listed below are references from these agencies regarding coastal erosion in Togo and coastal erosion as a world wide phenomena.  

- *"West Africa Coastal Erosion Project launched in Togo" (2016) *  [- link](http://www.ndf.fi/news/west-africa-coastal-erosion-project-launched-togo)
- *Agreement to Erosion Adaptation Project* (2016)  [- link](http://pubdocs.worldbank.org/en/493191479316551864/WACA-Lom%C3%A9-Communiqu%C3%A9-2016-English.pdf)
- World Bank WACA program brochure (2015)  [- link](http://pubdocs.worldbank.org/en/622041448394069174/1606426-WACA-Brochure.pdf)
- UNEP - Technologies for climate change adaption (2010)  [- link](http://www.unep.org/pdf/TNAhandbook_CoastalErosionFlooding.pdf)  
- USAID - Adapting to Coastal Climate Change (2009) -  [- link](http://www.crc.uri.edu/download/CoastalAdaptationGuide.pdf)  
- UNEP - Coastal Erosion and Climate Change in Western Africa(2002) -  [- link](http://www.unep.org/dewa/africa/publications/aeo-1/121.htm)
  
-------
### Algorithmic Profile:  
  
- This algorithm generates a water change product.
- The product is derived from Landsat7 Collection 1, Tier 2 sr imagery taken from USGS data holdings.    
- Utilizes WOFS for water detection.
-------  
### Process:  

- Load in a year's worth of data for years `y1` and `y2`, 
- Reduce noise by temporal compositing of both `y1` and `y2` to yield `y1_composite` and `y2_composite` .(This removes clouds, scanline errors, and statistical anomalies)      
- Run WOFS water classifier on `y1_composite` and `y2_composite`, to yield `y1_water` and `y2_water`. These outputs contain a 1 for water, and 0 for not water.     
- Subtract `y1_water` from `y2_water` to yield a `coastal_change` product. Where 0 represents no change, 1 represents water gain, and -1 represents water loss. 
*****  

> **Flow Diagram**
![](diagrams/rainy_demo/algo_nb3.png)

<hr>

# Index

* [Import Dependencies and Connect to the Data Cube](#coastal_change_classifier_import)
* [Choose Platform and Product](#coastal_change_classifier_plat_prod)
* [Define the Extents of the Analysis](#coastal_change_classifier_define_extents) (selecting too much can make the acquisition process slow)
* [Load Data from the Data Cube and Create Composites](#coastal_change_classifier_retrieve_data)
* [Obtain Water Classifications and Coastal Change](#coastal_change_classifier_water_cls_and_coastal_change)

## <span id="coastal_change_classifier_import">Import Dependencies and Connect to the Data Cube [&#9652;](#coastal_change_classifier_top)</span>

In [ ]:
import sys
import os
sys.path.append(os.environ.get('NOTEBOOK_ROOT'))

%matplotlib inline

from datetime import datetime
import numpy as np

import utils.data_cube_utilities.dc_utilities as utils

from utils.data_cube_utilities.clean_mask import landsat_qa_clean_mask
from utils.data_cube_utilities.dc_mosaic import create_mosaic#, ls8_unpack_qa
from utils.data_cube_utilities.dc_water_classifier import wofs_classify

from datacube.utils.aws import configure_s3_access
configure_s3_access(requester_pays=True)

# Initialize data cube object
import datacube
dc = datacube.Datacube(app='dc-coastal-erosion')

In [ ]:
start_time = datetime.now()
print("Start time: " + str(start_time))

## <span id="coastal_change_classifier_plat_prod">Choose Platform and Product [&#9652;](#coastal_change_classifier_top)</span>

In [ ]:
# Set query parameters
platform = 'LANDSAT_8'
product  = 'ls8_usgs_sr_scene'
collection = 'c1'
level = 'l2'

## <span id="coastal_change_classifier_define_extents">Define the Extents of the Analysis [&#9652;](#coastal_change_classifier_top)</span>

In [ ]:
# Select minimum and maximum longitudes and latitudes and the time range.

# Ghana
# Coastline east of Accra
lon = (0.0520, 0.3458)
lat = (5.6581, 5.8113)
first_year = 2013
first_time_range = (f'{first_year}-01-01', f'{first_year}-12-31')
last_year = 2014
second_time_range = (f'{last_year}-01-01', f'{last_year}-12-31')

**Visualize the selected area**

In [ ]:
from utils.data_cube_utilities.dc_display_map import display_map
display_map(lat, lon)

## <span id="coastal_change_classifier_retrieve_data">Load Data from the Data Cube and Create Composites [&#9652;](#coastal_change_classifier_top)</span>

In [ ]:
measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']
common_load_params = dict(platform=platform, product=product,
                          lon=lon, lat=lat, measurements=measurements,
                          dask_chunks={'time':1, 'latitude':1000, 'longitude':1000})

In [ ]:
# Retrieve data from Data Cube
first_dataset = dc.load(time=first_time_range, **common_load_params)

In [ ]:
# Retrieve data from Data Cube
second_dataset = dc.load(time=second_time_range, **common_load_params)

In [ ]:
from utils.data_cube_utilities.dc_utilities import ignore_warnings

plt_col_lvl_params = dict(platform=platform, collection=collection, level=level)

# Only keep pixels that are clear or have water.
first_clean_mask = landsat_qa_clean_mask(first_dataset, **plt_col_lvl_params)
second_clean_mask = landsat_qa_clean_mask(second_dataset, **plt_col_lvl_params)

# Remove noise from images by using appropriate data within the dataset to replace "dirty" data.
first_mosaic = ignore_warnings(create_mosaic, first_dataset, clean_mask=first_clean_mask)
second_mosaic = ignore_warnings(create_mosaic, second_dataset, clean_mask=second_clean_mask)

## <span id="coastal_change_classifier_water_cls_and_coastal_change">Obtain Water Classifications and Coastal Change [&#9652;](#coastal_change_classifier_top)</span>

In [ ]:
first_water_class = \
    ignore_warnings(wofs_classify, first_mosaic, mosaic=True, no_data=np.nan)
second_water_class = \
    ignore_warnings(wofs_classify, second_mosaic, mosaic=True, no_data=np.nan)

first_wofs = first_water_class.wofs.values
second_wofs = second_water_class.wofs.values

In [ ]:
coastal_change = second_water_class - first_water_class

In [ ]:
import matplotlib.pyplot as plt 
def plot_data_array_with_aspect(da):
    fig, ax = plt.subplots(figsize=(15,8))
    ax.set_aspect('equal')
    da.plot()

**Show the coastal change**

In [ ]:
# -1 -> water to coast
#  0 -> no change
#  1 -> coast to water (Coastal Erosion)

plot_data_array_with_aspect( coastal_change.wofs )

**Obtain the coastlines for the first and second time periods**

In [ ]:
first_coastline = np.zeros(first_wofs.shape)

for i in range(first_wofs.shape[0]):
    for j in range(first_wofs.shape[1]):
        pixel = first_wofs[i,j]
        if pixel == 0 and np.nansum(first_wofs[i-1:i+2, j-1:j+2]) >= 1 and np.nansum(first_wofs[i-1:i+2, j-1:j+2]) <= 5:
            first_wofs[i,j] = 1
            
second_coastline = np.zeros(second_wofs.shape)

for i in range(second_wofs.shape[0]):
    for j in range(second_wofs.shape[1]):
        pixel = second_wofs[i,j]
        if pixel == 0 and np.nansum(second_wofs[i-1:i+2, j-1:j+2]) >= 1 and np.nansum(second_wofs[i-1:i+2, j-1:j+2]) <= 5:
            second_wofs[i,j] = 1

**Show the water classifications for the first and second time periods**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
from pylab import imshow

In [ ]:
fig = plt.figure(figsize =  (15,8))
a=fig.add_subplot(1,2,1) 

imgplot = plt.imshow(first_wofs, cmap='Blues',
                     extent=[first_water_class.longitude.values.min(), 
                             first_water_class.longitude.values.max(),
                             first_water_class.latitude.values.min(),
                             first_water_class.latitude.values.max()])
a.set_title(f'wofs - {first_year}')
plt.colorbar(ticks=[0,1], orientation ='horizontal')
a=fig.add_subplot(1,2,2)
imgplot = plt.imshow(second_wofs, cmap='Blues',
                     extent=[second_water_class.longitude.values.min(), 
                             second_water_class.longitude.values.max(),
                             second_water_class.latitude.values.min(),
                             second_water_class.latitude.values.max()])
#imgplot.set_clim(0.0,1.0)
a.set_title(f'wofs - {last_year}')
plt.colorbar(ticks=[0,1], orientation='horizontal')
plt.savefig('wofs_compare.png')

**Show the coastline for the first time period**

In [ ]:
fig = plt.figure(figsize =  (15,8))
plt.imshow(first_coastline, cmap='Greens')

**Show the coastline for the second time period**

In [ ]:
fig = plt.figure(figsize =  (15,8))
plt.imshow(second_coastline, cmap='Purples')